### **API SADVR - Portrait statistique**  
https://www.cen.umontreal.ca/espacedoc/sadvr/  

Ce NoteBook est destiné à l'extraction et la visualisation de statistiques relatives aux professeur·e·s et à leurs expertises à partir de l'API de la vitrine de la recherche (SADVR). 
Celles-ci seront intégrées en un tableau de bord [PowerBI](https://wiki.umontreal.ca/display/SIE/Power+BI) permettant d'avoir un portrait d'ensemble des données.  

---

In [2]:
import pandas as pd
from SADVR_utils import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from rdflib import *

**Chargement des données**

In [3]:
data = updateInfoProfs()

**Expertises de recherche**

- Affiliations (facultés, départements, établissements affiliés)
- Disciplines
- Secteur de recherche
- Périodes chronologiques étudiées
- Régions géographiques / pays étudiés

In [4]:
expertises = data[['idsadvr', 'affiliations', 'etablissementsAffilies', 'expertise']]

toNormalize = ['affiliations', 'etablissementsAffilies', 'expertise', 'expertise.secteursRecherche',
                'expertise.disciplines', 'expertise.pays', 'expertise.continents', 'expertise.periodesChronologiques']

for c in toNormalize:
    expertises = explodeNormalize(expertises, c)

drop = ['affiliations.courrielInstitutionnel', 'affiliations.immeuble', 'affiliations.fonction.codeSad', 
        'affiliations.fonction.nom', 'affiliations.local', 'affiliations.exclusion', 'affiliations.exclusionTel',
        'affiliations.uniteAdministrative.codeSad', 'affiliations.uniteAdministrative.nom', 'affiliations.telephone.numero',
        'affiliations.telephone.poste', 'expertise.phraseCle']

expertises = expertises.drop(columns=drop)

*Établissements affiliés*

In [5]:
def groupEtablissements(etablissementNom : str) -> str : 
   return etablissementNom.split(' – ')[0]

In [6]:
# Etablissements affiliés
etablissementsAffilies = expertises.dropna(subset='etablissementsAffilies.nom')
etablissementsAffilies = etablissementsAffilies.drop_duplicates(subset=(['idsadvr', 'etablissementsAffilies.nom']))
etablissementsAffilies.loc[:, 'etablissementsAffilies.nom'] = etablissementsAffilies['etablissementsAffilies.nom'].apply(lambda x : x.split(' – ')[0])
etablissementsAffilies = pd.DataFrame(plotVariable(etablissementsAffilies, 'etablissementsAffilies.nom')).sort_values(by='count', ascending = False)
etablissementsAffilies = groupOtherValues(etablissementsAffilies, 10).reset_index(drop=True)

etablissementsAffilies

,etablissementsAffilies.nom,count
0,Centre hospitalier de l’Université de Montréal...,185
1,CIUSSS de l'Est-de-l'Île-de-Montréal,92
2,CIUSSS du Nord-de-l'Île-de-Montréal,83
3,Centre hospitalier universitaire Sainte-Justin...,77
4,CIUSSS du Centre-Sud-de-l’Île-de-Montréal,49
5,Institut de cardiologie de Montréal (ICM),33
6,Institut de recherches cliniques de Montréal (...,20
7,CISSS de Laval,9
8,Institut Philippe-Pinel de Montréal (IPP),8
9,Institut national de santé publique du Québec ...,7


In [7]:
px.pie(
    etablissementsAffilies,
    names = etablissementsAffilies['etablissementsAffilies.nom'],
    values = etablissementsAffilies['count'],
    hole = 0.6,
    title = 'Nombre de professeurs par établissements affiliés'
)

**Facultés, départements et établissements affiliés**

*Nombre de professeurs par facultés*

In [81]:
facultes = pd.DataFrame(plotVariable(expertises, 'affiliations.faculte.nom'))[:-2].sort_values(by='count')
facultes.to_csv('tables/statistiques__expertisesRecherche/profsParFaculte.csv', index=False)

px.pie(
    facultes, 
    values= facultes['count'], 
    names= facultes['affiliations.faculte.nom'], 
    title='Nombre de professeur-e-s par facultés',
    hole=0.6 
)

OSError: Cannot save file into a non-existent directory: 'tables\demographics'

**Expertises de recherche: Principales disciplines de recherche par faculté**

In [16]:
mappingDisciplines = pd.read_csv('tables/SADVR_disciplines.csv')
mappingDisciplines = mappingDisciplines[mappingDisciplines['noms.codeLangue'] == 'fre']
mappingDisciplines = {str(x['id']): x['noms.nom'] for x in mappingDisciplines.to_dict('records')}

In [79]:
disciplines = expertises[['idsadvr', 'affiliations.faculte.nom', 'expertise.disciplines.uid']].dropna(subset='expertise.disciplines.uid').drop_duplicates()
disciplines['expertise.disciplines.nom'] = disciplines['expertise.disciplines.uid'].astype(str).map(mappingDisciplines)
disciplines = disciplines.drop(columns='expertise.disciplines.uid')

# Group by 'faculty' and 'discipline' and count the number of professors in each group
faculty_discipline_counts = disciplines.groupby(['affiliations.faculte.nom', 'expertise.disciplines.nom'])['idsadvr'].count().reset_index()

# Rename the 'id' column to 'professor_count' for clarity
faculty_discipline_counts = faculty_discipline_counts.rename(columns={'id': 'professor_count'})

faculty_discipline_counts.to_csv('tables/statistiques__expertisesRecherche/disciplinesParFaculte.csv', index=False)
faculty_discipline_counts

,affiliations.faculte.nom,expertise.disciplines.nom,idsadvr
0,Faculté de droit,Administration de la santé,1
1,Faculté de droit,Administration publique,2
2,Faculté de droit,Communication,3
3,Faculté de droit,Criminologie,1
4,Faculté de droit,Droit,71
...,...,...,...
654,École de santé publique,Toxicologie,6
655,École de santé publique,Économie,5
656,École de santé publique,Éducation,1
657,École de santé publique,Épidémiologie et biostatistique,51


**Expertises de recherche: cartographie des expertises par mots-clés**

- Mots-clés / Phases clés associés aux disciplines / facultés / départements de recherche 

Voir si on peut extraire un graphe et le visualiser

*Disciplines*

In [ ]:
disciplines = expertises[[x for x in expertises.columns if 'discipline' in x or 'motsCles' in x or 'phraseCle' in x or x == 'idsadvr']]
disciplines

,idsadvr,expertise.motsCles,expertise.disciplines.uid,expertise.disciplines.codeLangue,expertise.disciplines.nom,expertise.disciplines.ordre
0,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",19,fre,Biochimie,2
1,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",20,fre,Biologie cellulaire,1
2,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",22,fre,Biologie moléculaire,3
3,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",19,eng,Biochemistry,2
4,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",20,eng,Cell Biology,1
...,...,...,...,...,...,...
285318,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",77,eng,Computer Science,1
285319,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",78,eng,Administrative Computing,2
285320,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",83,eng,Management,3
285321,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",86,eng,Pure Mathematics,4


In [ ]:
freqDisciplines = pd.DataFrame(plotVariable(disciplines, 'expertise.disciplines.uid', mapping=mappingDisciplines))
freqDisciplines

,expertise.disciplines.uid,count,mapping
0,98,196,Neurosciences
1,133,194,Sociologie
2,121,180,Psychologie
3,22,179,Biologie moléculaire
4,155,167,Santé publique
...,...,...,...
201,200,1,Oncologie gynécologique
202,65,1,Génie industriel
203,194,1,Mycologie
204,219,1,Anatomie


In [ ]:
# 1 premier exemple: Neurosciences
# Une fois que ça aura fonctionné pour une discipline, on va créer une fonction qui pourra le faire pour toutes les disciplines
neuro = disciplines[disciplines['expertise.disciplines.uid'] == '98']
neuro = explodeNormalize(neuro, 'expertise.motsCles')
neuro = neuro.dropna(subset='expertise.motsCles.nom')
neuro = neuro[neuro['expertise.motsCles.ordre'].astype(int) <= 5]

neuro = neuro.drop_duplicates(subset=['idsadvr', 'expertise.motsCles.uid'])
neuro = neuro [['idsadvr',	'expertise.disciplines.uid', 'expertise.disciplines.nom', 'expertise.motsCles.uid',
                'expertise.motsCles.nom',	'expertise.motsCles.ordre',	'expertise.motsCles.codeLangue']]

freqMotsCles = plotVariable(neuro, 'expertise.motsCles.uid').to_dict('records')
freqMotsCles = {x['expertise.motsCles.uid'] : x['count'] for x in freqMotsCles}

neuro['expertises.motCles.count'] = neuro['expertise.motsCles.uid'].map(freqMotsCles)
neuro = neuro.drop_duplicates(subset = ['expertise.motsCles.uid']).drop(columns='idsadvr')

neuro = neuro.sort_values(by='expertises.motCles.count', ascending=False).reset_index(drop=True)
neuro

,expertise.disciplines.uid,expertise.disciplines.nom,expertise.motsCles.uid,expertise.motsCles.nom,expertise.motsCles.ordre,expertise.motsCles.codeLangue,expertises.motCles.count
0,98,Neurosciences,98,Communication neuronale et neurotransmission,2,fre,25
1,98,Neurosciences,95,Cognition,1,fre,17
2,98,Neurosciences,5386,Neuroscience cognitive,2,fre,16
3,98,Neurosciences,1,Activité locomotrice / troubles du mouvement,1,fre,12
4,98,Neurosciences,5348,Imagerie cérébrale,3,fre,12
...,...,...,...,...,...,...,...
349,98,Neurosciences,505,Syntaxe,4,fre,1
350,98,Neurosciences,575,Troubles du langage chez l'enfant,5,fre,1
351,98,Neurosciences,204,Famille et aidants naturels,3,fre,1
352,98,Neurosciences,199,Évolution et phylogénie,3,fre,1


*On va essayer de créer un graphe avec*

In [ ]:
graph = neuro[:30]
listeMotsCles = graph.to_dict('records')

idDiscipline = str(98)

uriDisciplines = URIRef(f'{baseURI}/disciplines')
uriDiscipline = URIRef(f'{baseURI}/disciplines/{idDiscipline}')
uriCount = URIRef(f'{baseURI}/count')
uriHasKW = URIRef(f'{baseURI}/hasKeyword')

g = Graph()
g.bind('sadvr', 'https://www.recherche.umontreal.ca/vitrine/rest/api/1.7/umontreal/')

# Classe pour les disciplines
g.add((uriDisciplines, RDF.type, OWL.Class))
g.add((uriDisciplines, RDFS.label, Literal('Discipline', lang='fr')))

# Classe et nom de la discipline (fr)
g.add((uriDiscipline, RDF.type, uriDisciplines))
g.add((uriDiscipline, RDF.type, OWL.Class))
g.add((uriDiscipline, RDFS.label, Literal(mappingDisciplines[idDiscipline], lang='fr')))

# Relations
# keyWordCount
g.add((uriCount, RDF.type, OWL.ObjectProperty))
g.add((uriCount, RDFS.label, Literal('keywordCount', lang='en')))

for kw in listeMotsCles:
    id = kw['expertise.motsCles.uid']
    uri = URIRef(f'{baseURI}/motsCles/{id}')
    label = Literal(kw['expertise.motsCles.nom'])
    compte = Literal(kw['expertises.motCles.count'], datatype=XSD.integer)

    # Lien discipline <> mot-clé
    g.add((uri, RDFS.subClassOf, uriDiscipline))

    # Construction du noeud pour le mot-clé
    g.add((uri, RDF.type, OWL.Class))
    g.add((uri, RDFS.label, label))
    g.add((uri, uriCount, compte))

In [ ]:
g.serialize('visualisations/graph_test_neurosciences.ttl', format='ttl')

<Graph identifier=N3f3bc6de93e245d4a9a4409b5eda4950 (<class 'rdflib.graph.Graph'>)>

**Rayonnement académique**

- Publications (Nombre de publications par Faculté - à travers le temps ? - format des notices peu standard)